In [2]:
import pandas as pd
import numpy as np
import requests
import time
import json

In [3]:
df_rx = pd.read_csv('./Data/ohio_naloxone_pharmacies.csv')

FileNotFoundError: File b'./Data/ohio_naloxone_pharmacies.csv' does not exist

In [5]:
df_rx_cinci = df_rx[df_rx.CountyName=="Hamilton"]

In [6]:
df_rx_cinci.shape

(117, 5)

In [7]:
df_rx_cinci.head()

,CredentialNumber,Phone,BusinessName,Address,CountyName
704,22463050,(513) 865-1184,BETHESDA NORTH APOTHECARY,"10500 MONTGOMERY RD\nCINCINNATI, OH 45242",Hamilton
705,22570650,(513) 671-8603,CVS PHARMACY #16422,"900 E KEMPER RD\nSPRINGDALE, OH 45246",Hamilton
706,22571200,(513) 631-5717,CVS PHARMACY #16779,"4825A MARBURG AVE.\nCINCINNATI, OH 45209",Hamilton
707,22571300,(513) 719-0038,CVS PHARMACY #16873,"9040 COLERAIN AVE.\nCINCINNATI, OH 45251",Hamilton
708,22572450,(513) 719-1077,CVS PHARMACY #17534,"6150 GLENWAY AVE.\nCINCINNATI, OH 45211",Hamilton


In [1]:
df_rx_cinci['Address'].type()

NameError: name 'df_rx_cinci' is not defined

In [89]:
# Single address example
parameters = {
    "address":'17 WILLIAM H. TAFT ROAD, CINCINNATI, OH 45219',#"21 Marina Rd., Chelmsford, MA 01824",#"4600 Silver Hill Rd, Suitland, MD 20746",
    "format": "json",
    "searchtype": "onelineaddress",
    "benchmark":9,
    "vintage": "Census2010_Census2010",
    "layer": 14
}
#response = requests.get("https://geo.fcc.gov/api/census/block/find", params=parameters)
#response.content
response = requests.get("https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress", params=parameters)
data=response.json()
data
#data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["GEOID"][:12]
#data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLAT"].encode("ascii"), data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLON"].encode("ascii")

{u'result': {u'addressMatches': [],
  u'input': {u'address': {u'address': u'17 WILLIAM H. TAFT ROAD, CINCINNATI, OH 45219'},
   u'benchmark': {u'benchmarkDescription': u'Public Address Ranges - Census 2010 Benchmark',
    u'benchmarkName': u'Public_AR_Census2010',
    u'id': u'9',
    u'isDefault': False},
   u'vintage': {u'id': u'910',
    u'isDefault': True,
    u'vintageDescription': u'Census2010 Vintage - Census2010 Benchmark',
    u'vintageName': u'Census2010_Census2010'}}}}

In [78]:
address_to_censusblock("4600 Silver Hill Rd, Suitland, MD 20746")

[u'240338024051', {'x': '+38.8477828', 'y': '-076.9266698'}]

In [99]:
def address_to_censusblock(address_string):
    """Converts address to 12 digit census block group ID using 
    census geocoder API and returns list of ID and coordinates dictionary"""

    endpoint_url = "https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress"
    census_block = -1
    coordinates = {"x": -1000., "y": -1000.}
    try:
        parameters = {
            "address": address_string,
            "format": "json",
            "searchtype": "onelineaddress",
            "benchmark":9,
            "vintage": "Census2010_Census2010",
            "layer": 14
        }
        response = requests.get(endpoint_url, params=parameters)
        data = response.json()
        census_block = data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["GEOID"][:12].encode("ascii")
        coordinates = {
            "x": data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLAT"].encode("ascii"), 
            "y": data["result"]["addressMatches"][0]["geographies"]["Census Blocks"][0]["CENTLON"].encode("ascii")
        }
    except:
        print response.status_code
        
    return [census_block, coordinates]

In [100]:
start = time.time()
block_list = []
x_list = []
y_list = []
idx = 0
for address in df_rx_cinci.Address:
    address= address.replace('\n', ', ')
    [census_block, coords_dict] = address_to_censusblock(address)
    block_list.append(census_block)
    x_list.append(coords_dict["x"])
    y_list.append(coords_dict["y"])
    idx += 1
    if idx%500 == 0:
        print idx, "coordinates converted"
    if census_block == -1:
        print address
print "total time:",(time.time() - start)/60., "mins"

200
17 WILLIAM H. TAFT ROAD, CINCINNATI, OH 45219
200
8372 VINE STREET, CINCINNATI, OH 45216
200
4001 S.R. #128, CLEVES, OH 45002
200
7385 WOOSTER ROAD, MARIEMONT, OH 45227
200
8241 VINE STREET, CINCINNATI, OH 45216
200
3711 STONE CREEK BLVD, CINCINNATI, OH 45251
200
EDEN & ALBERT SABIN WAY, ML #443, CINCINNATI, OH 45219
200
2801 CUNNINGHAM DRIVE, CINCINNATI, OH 45241
200
9520 FIELDS ERTEL ROAD, LOVELAND, OH 45140
200
719 OHIO PIKE, CINCINNATI, OH 45245
200
2203 BEECHMONT AVE, CINCINNATI, OH 45230
total time: 2.92984999816 mins


In [111]:
tract_list = [str(block)[:11] for block in block_list]

In [112]:
df_rx_cinci["CENSUS_BLOCK_ID"] = block_list
df_rx_cinci["CENSUS_TRACT_ID"] = tract_list
df_rx_cinci["LAT_X"] = x_list
df_rx_cinci["LON_y"] = y_list

C:\Users\isheu\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\isheu\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\isheu\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [117]:
# Filter out addresses that could not be mapped
df_rx_cinci[df_rx_cinci.CENSUS_BLOCK_ID==-1].shape
df_rx_cinci_final = df_rx_cinci[df_rx_cinci.CENSUS_BLOCK_ID != -1]

In [118]:
# Save to file
print df_rx_cinci_final.shape
df_rx_cinci_final.to_csv('./Data/pharmacies_cinci_processed.csv', index=False)

(106, 9)
